In [1]:
# for compatible with python 3
from __future__ import print_function
import os
# os.environ["KERAS_BACKEND"] = "theano"
import numpy as np
from utils.data import read_stock_history, index_to_date, date_to_index, normalize
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
from model.ddpg.actor import ActorNetwork
from model.ddpg.critic import CriticNetwork
from model.ddpg.ddpg import DDPG
from model.ddpg.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

import numpy as np
import tflearn
import tensorflow as tf

from stock_trading import StockActor, StockCritic, obs_normalizer, get_model_path, get_result_path, \
                          test_model, get_variable_scope, test_model_multiple, convert_R_output
    
from model.supervised.lstm import StockLSTM
from model.supervised.cnn import StockCNN

from environment.portfolio import PortfolioEnv, MultiActionPortfolioEnv

Using TensorFlow backend.


In [5]:
# parameters calculated in R
filepath = '/Users/zachariemartin/Desktop/School/Projects/summer2019/2/sum19/'

alpha, beta, omega, Q_bar = convert_R_output(coef_path = filepath + 'coef.csv',\
                                                   Q_bar_path = filepath + 'Q_bar.csv', num_assets=10)

In [6]:
# common settings
batch_size = 64
action_bound = 1.
tau = 1e-3

models = []
model_names = []
window_length_lst = [3]
predictor_type_lst = ['cnn']
use_batch_norm = True

In [7]:
import pandas as pd
# # param dict
# parameters = {}

# # 10 assets
num_assets = 10
num_rows_per_asset = 5

# # calculated in R
# a = 0.007073296
# b = 0.658588119

# # import from R
coef = pd.read_csv(filepath + 'coef.csv')

# # mu, ar1, omega, alpha, beta 0,1,2,3,4 for each asset - 5 values for each asset so 500 total values
# parameters['alpha'] = np.array([coef.loc[i,'x'] for i in range(3,(num_assets*num_rows_per_asset),5)])
# parameters['beta'] = np.array([coef.loc[i,'x'] for i in range(4,(num_assets*num_rows_per_asset),5)])
# parameters['omega'] = np.array([coef.loc[i,'x'] for i in range(2,(num_assets*num_rows_per_asset),5)])
# parameters['Q_bar'] = pd.read_csv('Q_bar.csv').drop('Unnamed: 0',axis=1).to_numpy()
# parameters['H_init'] = pd.read_csv('H_init.csv').drop('Unnamed: 0',axis=1).to_numpy()
# parameters['Q'] = pd.read_csv('Q_init.csv').drop('Unnamed: 0',axis=1).to_numpy()
# parameters['T'] = 1000
# parameters['small_scalar'] = 1e-5
# parameters['num_assets'] = 10

parameters = {
                'alpha' : np.array([coef.loc[i,'x'] for i in range(3,(num_assets*num_rows_per_asset),5)]),
                'beta' : np.array([coef.loc[i,'x'] for i in range(4,(num_assets*num_rows_per_asset),5)]),
                'omega' : np.array([coef.loc[i,'x'] for i in range(2,(num_assets*num_rows_per_asset),5)]),
                'Q_bar' : pd.read_csv(filepath + 'Q_bar.csv').drop('Unnamed: 0',axis=1).to_numpy(),
                'H_init' : pd.read_csv(filepath + 'H_init.csv').drop('Unnamed: 0',axis=1).to_numpy(),
                'Q' : pd.read_csv(filepath + 'Q_init.csv').drop('Unnamed: 0',axis=1).to_numpy(),
                'T' : 1000,
                'small_scalar' : 1e-5,
                'num_assets' : 10,
                # calculated in R - parameters for Q process
                'a' : 0.007073296,
                'b' : 0.658588119,
                'mean' : 100
}

env = PortfolioEnv(parameters)


Data generated


In [8]:
nb_classes = parameters['num_assets'] + 1

In [9]:
for window_length in window_length_lst:
    for predictor_type in predictor_type_lst:
        name = 'DDPG_window_{}_predictor_{}'.format(window_length, predictor_type)
        model_names.append(name)
        tf.reset_default_graph()
        sess = tf.Session()
        tflearn.config.init_training_mode()
        action_dim = [nb_classes]
        state_dim = [nb_classes, window_length]
        variable_scope = get_variable_scope(window_length, predictor_type, use_batch_norm)
        with tf.variable_scope(variable_scope):
            actor = StockActor(sess, state_dim, action_dim, action_bound, 1e-4, tau, batch_size, predictor_type, 
                               use_batch_norm)
            critic = StockCritic(sess=sess, state_dim=state_dim, action_dim=action_dim, tau=1e-3,
                                 learning_rate=1e-3, num_actor_vars=actor.get_num_trainable_vars(), 
                                 predictor_type=predictor_type, use_batch_norm=use_batch_norm)
            actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim))

            model_save_path = get_model_path(window_length, predictor_type, use_batch_norm)
            summary_path = get_result_path(window_length, predictor_type, use_batch_norm)

            ddpg_model = DDPG(env, sess, actor, critic, actor_noise, obs_normalizer=obs_normalizer,
                              config_file='config/stock.json', model_save_path=model_save_path,
                              summary_path=summary_path)
            ddpg_model.initialize(load_weights=False, verbose=False)
            models.append(ddpg_model)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Build model from scratch


In [10]:
ddpg_model.train()

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'
Number of episodes:  500
predicted_q_value.shape (64, 1)
j: 100, ep_ave_max_q: 0.014123039727564901
predicted_q_value.shape (64, 1)
j: 200, ep_ave_max_q: 0.09391428963863291
predicted_q_value.shape (64, 1)
j: 300, ep_ave_max_q: 0.26812426539254375
predicted_q_value.shape (64, 1)
j: 400, ep_ave_max_q: 0.6527520804957021
predicted_q_value.shape (64, 1)
j: 500, ep_ave_max_q: 1.1018295937974472
predicted_q_value.shape (64, 1)
j: 600, ep_ave_max_q: 1.5843409148801584
predicted_q_value.shape (64, 1)
j: 700, ep_ave_max_q: 2.1082949948322494
[EPISODE]: 0, [REWARD]: -0.20779894, [Qmax]: 0.00311466
predicted_q_value.shape (64, 1)
j: 0, ep

KeyboardInterrupt: 